# Test RAG

In [1]:
import os
import bs4
from dotenv import load_dotenv

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_postgres import PGVector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [15]:
load_dotenv(override=True)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

connection_string = DATABASE_URL.replace("postgresql+psycopg2://", "postgresql+psycopg://")

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="customer_support_vector_db",
    connection=connection_string,
    use_jsonb=True,
    engine_args={
        "pool_size": 1,
        "max_overflow": 0,
        "pool_recycle": 300,
        "pool_pre_ping": True,
        "connect_args": {
            "connect_timeout": 10,
            "keepalives": 1,
            "keepalives_idle": 30,
            "keepalives_interval": 10,
            "keepalives_count": 5,
        }
    }
)


retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [57]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile",
    temperature=0.2
)

In [58]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. 

Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
IMPORTANT: Keep the standalone question in the SAME LANGUAGE as the user's latest question.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)


qa_system_prompt = """You are a helpful and friendly Customer Assistant named 'Mimin'.
Use the following pieces of retrieved context to answer the question.

RULES:
1. Answer in **Indonesian Language (Bahasa Indonesia)**, unless the user specifically asks in English.
2. If the context does not contain the answer, say "Waduh, maaf kak, Mimin gak nemu infonya di catatan." (Don't make up answers).
3. Keep the answer concise (max 3 sentences) but polite.
4. If the user greets you (Hi, Halo), reply nicely in Indonesian.

Context:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [59]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [60]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Tes

In [61]:
session_id = "user_sawit_123"

In [62]:
def tanya_bot(tanya):
    print(f"User: {tanya}")
    print("Bot mikir...", end="\r")
    
    response = conversational_rag_chain.invoke(
        {"input": tanya},
        config={"configurable": {"session_id": session_id}}
    )["answer"]
    
    print(f"Bot : {response}\n" + "-"*30)

In [65]:
tanya_bot("what is the refund policy?")

tanya_bot("How long does it take usually?")

tanya_bot("Do you have contact email?")

User: what is the refund policy?
Bot : Waduh, maaf kak, Mimin gak nemu infonya di catatan.
------------------------------
User: How long does it take usually?
Bot : Waduh, maaf kak, Mimin gak nemu infonya di catatan.
------------------------------
User: Do you have contact email?
Bot : Waduh, maaf kak, Mimin gak nemu infonya di catatan.
------------------------------
